In [ ]:
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
# Part 1: Preprocessing functions
def tolowercase(text):
    text=text.lower()
    return text

def remove_punctuation(text):
    new_text=""
    for j in range(len(text)):
        char=text[j]
        if char.isalnum() or char == " ":
            new_text+=char
    
    return new_text
        

def remove_stop_words(tokens, stop_words_list):
    for words in stop_words_list:
        if words in tokens:
            tokens.remove(words)

    return tokens


def tokenize(text):
   return text.split()

def preprocess_text(text,stop_words):
    text=tolowercase(text)
    text=remove_punctuation(text)
    tokens=tokenize(text)
    tokens=remove_stop_words(tokens,stop_words_list=stop_words)

    return tokens


# Part 2: Feature extraction
def build_vocabulary(documents,stop_words):
    vocabulary=set()
    for text in documents:
        tokens=preprocess_text(text,stop_words)
        vocabulary.update(tokens)

    vec=[*vocabulary]
    return vec

def create_bow_vector(document, vocabulary):
    bgw=[]
    
    for text in document:
        vec=[0]*len(vocabulary)
        tokens=tokenize(text)
        for word in tokens:
            if word in vocabulary:
                i=vocabulary.index(word)
                vec[i]+=1
                
        bgw.append(vec)

    return bgw


def convert_to_binary_features(bow_vectors):
    """Convert count features to binary features (0/1 for word presence/absence)"""
    
    for features in bow_vectors:
        for i in range(len(features)):
            if features[i]>=1:
                features[i]=1
            
    return bow_vectors



def preprocessing(X,stop_words):
    vocabulary=build_vocabulary(X,stop_words)
    bgw=create_bow_vector(X,vocabulary)
    bgw=convert_to_binary_features(bgw)
    return bgw,vocabulary


In [ ]:

def cross_entropy(Y):
    count=Counter(Y)
    P=[]
    for i in count.values():
        P.append(i/len(Y))
    P=np.array(P)
    return -np.sum(np.dot(P,np.log2(P)))


def calculate_gini(Y):
    """Calculate Gini impurity for a set of labels"""
    count=Counter(Y)
    P=[]
    for i in count.values():
        P.append(i/len(Y))
    P=np.array(P)

    return 1-np.sum(P**2)


def calculate_information_gain(parent, left_child, right_child):
    """Calculate information gain for a split"""
    Ep=calculate_gini(parent)    
    left_w=len(left_child)/parent.shape[0]
    right_w=len(right_child)/parent.shape[0]
    Ec=(left_w*calculate_gini(left_child)+right_w*calculate_gini(right_child))/2

    return Ep-Ec


def split(X,Y):

    for j in range(X.shape[1]):
        thresholds=np.unique(X[:,j]) #get all unique values as thresholds

    max_IG=float('-inf')
    left_split,right_split=[],[]
    left_split_l,right_split_l=[],[]
    best_feature_index,best_threshold=0,0

    for j in range(X.shape[1]):
        
            
        for threshold in thresholds:
            
            Left,Right=[],[]
            Left_l,Right_l=[],[]

            for i in range(X.shape[0]):
                    if X[i,j]<= threshold:
                        Left.append(X[i])
                        Left_l.append(Y[i])
                    else:
                        Right.append(X[i])
                        Right_l.append(Y[i])
    

            IG=calculate_information_gain(Y,Left_l,Right_l)
            if  IG>max_IG:
                max_IG=IG
                left_split,right_split=Left,Right
                left_split_l,right_split_l=Left_l,Right_l
                best_feature_index=j
                best_threshold=threshold
            

    return left_split,right_split,left_split_l,right_split_l,best_feature_index,best_threshold

        

In [ ]:
class DecisionTree:
    def __init__(self,threshold=None,label=None,feature_index=None):

        # node values to store
        self.feature_index=feature_index
        self.threshold=threshold
        self.label=label # label for leaf node for prediction

        #pointers
        self.left=None
        self.right=None


def build_tree(depth,n,X,Y):
    if len(Y) == 0:
        return None
    
    if all(Y==Y[0]) or Y.shape[0]==1:
        return DecisionTree(label=Y[0]) # check if all labels are same
    
    if depth==n:
        return DecisionTree(label=np.bincount(Y).argmax()) #returning the most common label
    
    left,right,left_l,right_l,feature_index,threshold=split(X,Y)

    

    node=DecisionTree(feature_index=feature_index,threshold=threshold)

    node.left=build_tree(depth+1,n,np.array(left),np.array(left_l))
    node.right=build_tree(depth+1,n,np.array(right),np.array(right_l))

    return node


def get_feature_importance(tree, feature_names,importance):
    """Identify important features based on tree structure"""
    
    if tree.left==None or tree.right==None:
        return importance

    feature = feature_names[tree.feature_index]
    importance.append(feature)  

    get_feature_importance(tree.left, feature_names, importance)
    get_feature_importance(tree.right, feature_names, importance)

    return importance


def train(X,Y,max_depth):
    return build_tree(0,max_depth,X,Y)


In [ ]:

def predict(X,Node):
    if Node.left==None or Node.right==None:
        return Node.label
    
    if X[Node.feature_index]<=Node.threshold:
        return predict(X,Node.left)
    else:
        return predict(X,Node.right)
    

def prediction(X,Tree):
    result_arr=[]
    for i in range(len(X)):
        result=predict(X[i],Tree)
        result_arr.append(int(result))
    return result_arr

def accurracy(y_pred,y_true):
    return np.mean(y_pred==y_true)



In [ ]:
# Sample data 
sample_positive = [
    "i love this product it works great",
    "excellent service and fast delivery",
    "best purchase i have made this year",
    "very happy with the quality of this item",
    "the customer service was amazing and helpful",
    "works perfectly and arrived on time",
    "exceeded my expectations highly recommend",
    "easy to use and does exactly what it says",
    "great value for money will buy again",
    "fantastic product and excellent experience"
]

sample_negative = [
    "terrible experience would not recommend",
    "product broke after two days",
    "customer service was unhelpful and rude",
    "waste of money does not work properly", 
    "extremely disappointed with this purchase",
    "arrived late and damaged",
    "did not match the description at all",
    "poor quality and overpriced",
    "worst product i have ever bought",
    "completely useless and frustrating"
]

# A minimal stop words list
stop_words = ["the", "a", "an", "and", "or", "but", "is", "are", "was", "were", 
              "i", "you", "he", "she", "it", "we", "they", "this", "that", "have", "has"]



X=sample_positive+sample_negative

X,vocabulary=preprocessing(X,stop_words)

labels = [0] * len(sample_positive) + [1] * len(sample_positive)

X_train, X_test, y_train, y_test = train_test_split(X,labels,test_size=0.2)

X_train=np.atleast_2d(X_train)
X_test=np.atleast_2d(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [ ]:
#training for decision tree

Tree=train(X_train,y_train,5)

In [ ]:
importance=[]
importance=get_feature_importance(Tree,vocabulary,importance)

In [ ]:
importance

In [ ]:
#testing the tree

y_pred=prediction(X_test,Tree)

print(y_pred)

In [ ]:
#accuracy 
acc=accurracy(y_pred,y_test)
print(f"accurracy is {acc}")

In [ ]:
#visualizing decision tree

import networkx as nx
import matplotlib.pyplot as plt

def add_edges(nx_tree,node,parent=None,edge_label=""):
    if node is None:
        return
    
    if node.label is not None:
        node_label = f"Label: {node.label}"
    else:
        node_label = f"{node.feature_index} <= {node.threshold}"

    nx_tree.add_node(id(node), label=node_label)

    if parent is not None:
        nx_tree.add_edge(parent, id(node), label=edge_label)

    add_edges(nx_tree, node.left, id(node), "Yes")
    add_edges(nx_tree, node.right, id(node), "No")
    
def draw_tree(Tree):
    nx_tree=nx.DiGraph()
    add_edges(nx_tree,Tree)
    pos = nx.spring_layout(nx_tree)  
    labels = nx.get_node_attributes(nx_tree, "label")
    edge_labels = nx.get_edge_attributes(nx_tree, "label")

    plt.figure(figsize=(10, 6))
    nx.draw(nx_tree, pos, with_labels=True, labels=labels, node_size=3000, node_color="lightblue", edge_color="black")
    nx.draw_networkx_edge_labels(nx_tree, pos, edge_labels=edge_labels)
    
    plt.show()

draw_tree(Tree)
